## 데이터 전처리

In [1]:
import pandas as pd

In [2]:
kospi50_daily_stock_chart = pd.read_csv("./data/kospi50-1980-2021.csv")
kospi50_daily_stock_chart

,code,name,date,open,high,low,close
0,U182,kospi50_index,20000302,804.739990,833.580017,786.229980,826.950012
1,U182,kospi50_index,20000303,843.460022,845.299988,816.580017,832.000000
2,U182,kospi50_index,20000306,844.330017,849.239990,831.119995,836.200012
3,U182,kospi50_index,20000307,831.460022,833.369995,816.719971,833.349976
4,U182,kospi50_index,20000308,823.349976,847.530029,810.869995,844.239990
...,...,...,...,...,...,...,...
276194,A005380,현대차,20210831,210000.000000,212500.000000,208000.000000,212500.000000
276195,A005380,현대차,20210901,214000.000000,218500.000000,213500.000000,215000.000000
276196,A005380,현대차,20210902,214000.000000,215000.000000,210000.000000,211500.000000
276197,A005380,현대차,20210903,213000.000000,215500.000000,212000.000000,214500.000000


In [3]:
def preprocess(stock_chart):
    z_feature = stock_chart[["code", "name", "date"]].values.tolist()
    stock_chart = stock_chart.values.tolist()
    stock_first_index = 0
    
    for idx, daily in enumerate(stock_chart):
        z_feature[idx].append(daily[3] / daily[6] - 1)    # open
        z_feature[idx].append(daily[4] / daily[6] - 1)    # high
        z_feature[idx].append(daily[5] / daily[6] - 1)    # low
        
        if stock_chart[idx-1][1] != daily[1]:
            s_idx = idx
            for i in range(7):
                z_feature[idx].append(0) # close
            continue
        else:
            z_feature[idx].append(daily[6] / stock_chart[idx-1][6] - 1) # close
        
        for k in [5, 10, 15, 20, 25, 30]: # z_dk
            if (idx-s_idx)-(k-1) < 0:
                sum_close = stock_chart[s_idx:idx+1]
            else:
                sum_close = stock_chart[idx-(k-1):idx+1]

            sum_close = [chart[6] for chart in sum_close]
            sum_close = sum(sum_close) / (len(sum_close) * daily[6]) - 1
            z_feature[idx].append(sum_close)

    return pd.DataFrame(z_feature, columns=["code", "name", "date", "open", "high", "low", "close",
                                            "z_d5", "z_d10", "z_d15", "z_d20", "z_d25", "z_d30"])

In [4]:
preprocessed_kospi50_daily_stock_chart = preprocess(kospi50_daily_stock_chart)
preprocessed_kospi50_daily_stock_chart

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,U182,kospi50_index,20000302,-0.026858,0.008017,-0.049241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,U182,kospi50_index,20000303,0.013774,0.015986,-0.018534,0.006107,-0.003035,-0.003035,-0.003035,-0.003035,-0.003035,-0.003035
2,U182,kospi50_index,20000306,0.009723,0.015594,-0.006075,0.005048,-0.005362,-0.005362,-0.005362,-0.005362,-0.005362,-0.005362
3,U182,kospi50_index,20000307,-0.002268,0.000024,-0.019956,-0.003408,-0.001470,-0.001470,-0.001470,-0.001470,-0.001470,-0.001470
4,U182,kospi50_index,20000308,-0.024744,0.003897,-0.039527,0.013068,-0.011480,-0.011480,-0.011480,-0.011480,-0.011480,-0.011480
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276194,A005380,현대차,20210831,-0.011765,0.000000,-0.021176,0.011905,-0.011294,-0.019059,-0.005176,0.008471,0.015247,0.023843
276195,A005380,현대차,20210901,-0.004651,0.016279,-0.006977,0.011765,-0.017674,-0.029302,-0.017984,-0.005233,0.001581,0.010078
276196,A005380,현대차,20210902,0.011820,0.016548,-0.007092,-0.016279,0.000000,-0.010875,-0.003940,0.008038,0.016076,0.024192
276197,A005380,현대차,20210903,-0.006993,0.004662,-0.011655,0.014184,-0.008392,-0.018648,-0.019270,-0.008042,0.000466,0.007692


In [6]:
samsung_stock_chart = preprocessed_kospi50_daily_stock_chart[preprocessed_kospi50_daily_stock_chart.name == "삼성전자"]
samsung_stock_chart

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
167885,A005930,삼성전자,19800104,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
167886,A005930,삼성전자,19800105,-0.048780,0.000000,-0.048780,0.025000,-0.012195,-0.012195,-0.012195,-0.012195,-0.012195,-0.012195
167887,A005930,삼성전자,19800107,-0.022727,0.000000,-0.022727,0.073171,-0.053030,-0.053030,-0.053030,-0.053030,-0.053030,-0.053030
167888,A005930,삼성전자,19800108,0.000000,0.000000,-0.021739,0.045455,-0.070652,-0.070652,-0.070652,-0.070652,-0.070652,-0.070652
167889,A005930,삼성전자,19800109,-0.020833,0.000000,-0.041667,0.043478,-0.087500,-0.087500,-0.087500,-0.087500,-0.087500,-0.087500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179048,A005930,삼성전자,20210831,-0.023468,0.000000,-0.031291,0.028150,-0.019817,-0.029335,-0.018861,0.002738,0.007927,0.011734
179049,A005930,삼성전자,20210901,-0.001302,0.003906,-0.011719,0.001304,-0.018229,-0.026823,-0.023090,-0.001563,0.005729,0.009462
179050,A005930,삼성전자,20210902,0.010526,0.010526,-0.003947,-0.010417,-0.004211,-0.012763,-0.015000,0.004408,0.014632,0.018991
179051,A005930,삼성전자,20210903,-0.002611,0.001305,-0.007833,0.007895,-0.006005,-0.015405,-0.023064,-0.007050,0.005431,0.009661


In [9]:
preprocessed_kospi50_daily_stock_chart.to_csv("./data/preprocessed_kospi50_daily_stock_chart.csv", index=False)
samsung_stock_chart.to_csv("./data/samsung_stock_chart.csv", index=False)

In [10]:
preprocessed_kospi50_daily_stock_chart.describe()

,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
count,2.761990e+05,276199.000000,276199.000000,276199.000000,276199.000000,276199.000000,276199.000000,276199.000000,276199.000000,276199.000000,276199.000000
mean,2.006647e+07,0.000242,0.016160,-0.015613,0.000769,-0.000087,-0.000228,-0.000393,-0.000554,-0.000702,-0.000842
std,1.037884e+05,0.023398,0.017453,0.024008,0.027219,0.029802,0.045200,0.056303,0.065717,0.074232,0.082008
min,1.980010e+07,-0.459641,-0.459641,-0.459641,-0.570056,-0.612259,-0.688792,-0.714302,-0.727058,-0.734711,-0.739813
25%,2.000052e+07,-0.010870,0.004673,-0.021376,-0.012438,-0.014575,-0.022702,-0.029011,-0.034331,-0.039314,-0.043680
50%,2.008113e+07,0.000000,0.011765,-0.011111,0.000000,0.000000,-0.000109,-0.000748,-0.001375,-0.001796,-0.002244
75%,2.015082e+07,0.011662,0.022140,-0.004474,0.012448,0.014019,0.020775,0.025672,0.029936,0.033461,0.036543
max,2.021091e+07,0.359223,0.440129,9.000974,3.261194,1.060707,1.193295,1.237491,1.259589,1.437411,1.691304
